In [1]:
#ecg_wave_model.py
import os
import sys

from tensorflow.keras.layers import Lambda, Input, Dense, LSTM, RepeatVector, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import mse, binary_crossentropy, kullback_leibler_divergence
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l1, l2, l1_l2
import numpy as np
from tensorflow.keras.layers import SpatialDropout1D, SpatialDropout2D, SpatialDropout3D, Lambda
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Activation, Flatten
from tensorflow.keras.layers import Conv1D, Conv2D, Conv3D, UpSampling2D, UpSampling3D, MaxPooling1D
from tensorflow.keras.layers import MaxPooling2D, MaxPooling3D, AveragePooling1D, AveragePooling2D, AveragePooling3D

# IPython imports
from IPython.display import Image

# ML4CVD Imports
from ml4cvd.arguments import parse_args
from ml4cvd.tensor_generators import test_train_valid_tensor_generators
from ml4cvd.defines import TENSOR_EXT, IMAGE_EXT, MRI_TO_SEGMENT, MRI_SEGMENTED
from ml4cvd.recipes import train_multimodal_multitask, test_multimodal_multitask
from ml4cvd.models import make_multimodal_multitask_model, train_model_from_generators
HD5_FOLDER = '/mnt/disks/ecg-rest-38k-tensors/2020-03-14/'

In [3]:
sys.argv = ['train', 
            '--tensors', HD5_FOLDER, 
            '--input_tensors', 'ecg_rest', 
            '--output_tensors', 'bmi_21', 
            '--batch_size', '32', 
            '--epochs', '1',
            '--training_steps', '30',
            '--id', 'ecg_regularized']
args = parse_args()
generate_train, generate_valid, generate_test = test_train_valid_tensor_generators(**args.__dict__)

2020-06-29 10:22:48,181 - logger:25 - INFO - Logging configuration was loaded. Log messages can be found at ./recipes_output/ecg_regularized/log_2020-06-29_10-22_0.log.
2020-06-29 10:22:48,183 - arguments:407 - INFO - Command Line was: 
./scripts/tf.sh train --tensors /mnt/disks/ecg-rest-38k-tensors/2020-03-14/ --input_tensors ecg_rest --output_tensors bmi_21 --batch_size 32 --epochs 1 --training_steps 30 --id ecg_regularized

2020-06-29 10:22:48,184 - arguments:408 - INFO - Total TensorMaps: 591 Arguments are Namespace(activation='relu', aligned_dimension=16, alpha=0.5, anneal_max=2.0, anneal_rate=0.0, anneal_shift=0.0, app_csv=None, b_slice_force=None, balance_csvs=[], batch_size=32, bigquery_credentials_file='/mnt/ml4cvd/projects/jamesp/bigquery/bigquery-viewer-credentials.json', bigquery_dataset='broad-ml4cvd.ukbb7089_r10data', block_size=3, bottleneck_type=<BottleneckType.FlattenRestructure: 1>, cache_size=875000000.0, categorical_field_ids=[], continuous_field_ids=[], continuous_

In [4]:
l1_weight = 0.0
l2_weight = 0.0
filters = 32
kernel_size = 71

In [8]:
input_x = Input(shape=(5000, 12), name=args.tensor_maps_in[0].input_name())
x = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu',
           kernel_regularizer=l1_l2(l1=l1_weight, l2=l2_weight),
           bias_regularizer=l1_l2(l1=l1_weight, l2=l2_weight),
          )(input_x)
x = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu',
           kernel_regularizer=l1_l2(l1=l1_weight, l2=l2_weight),
           bias_regularizer=l1_l2(l1=l1_weight, l2=l2_weight),
          )(x)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu',
           kernel_regularizer=l1_l2(l1=l1_weight, l2=l2_weight),
           bias_regularizer=l1_l2(l1=l1_weight, l2=l2_weight),
          )(x)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu',
           kernel_regularizer=l1_l2(l1=l1_weight, l2=l2_weight),
           bias_regularizer=l1_l2(l1=l1_weight, l2=l2_weight),
          )(x)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu',
           kernel_regularizer=l1_l2(l1=l1_weight, l2=l2_weight),
           bias_regularizer=l1_l2(l1=l1_weight, l2=l2_weight),
          )(x)
x = Flatten()(x)
x = Dense(filters, activation='relu')(x)
output_y = Dense(1, activation='linear', name=args.tensor_maps_out[0].output_name())(x)

m = Model(inputs=[input_x], outputs=[output_y])
m.summary()
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipnorm=1.0)
m.compile(optimizer=opt, loss='logcosh')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_strip_continuous (Inpu [(None, 5000, 12)]        0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 4930, 32)          27296     
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 4860, 32)          72736     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 2430, 32)          0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 2360, 32)          72736     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 1180, 32)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 1110, 32)          72736 

In [9]:
m = train_model_from_generators(m, generate_train, generate_valid, args.training_steps,
                                        args.validation_steps, args.batch_size, args.epochs, args.patience,
                                        args.output_folder, args.id, args.inspect_model, args.inspect_show_labels)

2020-06-29 10:27:14,190 - tensor_generators:151 - INFO - Started 2 train workers with cache size 0.875 GB.
2020-06-29 10:27:15,905 - tensor_generators:151 - INFO - Started 1 validation workers with cache size 0.875 GB.
Train for 30 steps, validate for 18 steps
29/30 [============================>.] - ETA: 0s - loss: 0.3110
Epoch 00001: val_loss improved from inf to 0.30896, saving model to ./recipes_output/ecg_regularized/ecg_regularized.h5
30/30 [==============================] - 22s 724ms/step - loss: 0.3078 - val_loss: 0.3090
2020-06-29 10:27:40,219 - tensor_generators:238 - INFO - Stopped 2 workers. 
2020-06-29 10:27:40,221 - tensor_generators:238 - INFO - Stopped 1 workers. 
2020-06-29 10:27:40,223 - models:1126 - INFO - Model weights saved at: ./recipes_output/ecg_regularized/ecg_regularized.h5
2020-06-29 10:27:41,298 - plots:215 - INFO - Saved learning curves at:./recipes_output/ecg_regularized/metric_history_ecg_regularized.png
